# Building a test dataset
## Introduction
- To run test, it's important to have a test dataset.
- We can use an LLM to generate a synthetic dataset.
- Here's we'll use RAGAS to generate a synthetic dataset.
## Installation

https://docs.ragas.io/en/stable/getstarted/rag_testset_generation/#load-documents

In [1]:
%pip install -q ragas langchain_openai langchain pypdf


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import sys
sys.path.append('../..')

## Setup embeddings and llm

In [9]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# Directly create instances of ChatOpenAI and OpenAIEmbeddings
llm = ChatOpenAI(model="gpt-4o")
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

generator_llm = LangchainLLMWrapper(llm)
generator_embeddings = LangchainEmbeddingsWrapper(embeddings)


## Load and index the documents

- Here we use a Webloader to load in some documents from the website.

In [ ]:
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from typing import List

# Load text from webpage
web_url = "https://docs.ragas.io/en/stable/"

loader = WebBaseLoader(web_url, show_progress=True, )
data = loader.load()

# Split text into documents
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
documents  = text_splitter.split_documents(data)
print(len(documents))

25


- To handle PDF documents, we use the PyPDFLoader.

In [5]:
from langchain_community.document_loaders import PyPDFLoader
pdf_url = "data/papers/microsoft_annual_report_2022.pdf"
loader = PyPDFLoader(pdf_url)

documents = []
pages = []
async for page in loader.alazy_load():
    pages.append(page)

print("PDF pages:", len(pages))

# Add PDF pages to the documents
documents.extend(pages)
print("Documents:", len(documents))

PDF pages: 93
Documents: 93


- Internally RAGAS uses `rapidfuzz` to calculate the string distance.

In [6]:
%pip install -q rapidfuzz


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [10]:
from ragas.testset import TestsetGenerator # type: ignore

# Number of questions to generate
testset_size = 10

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(documents, testset_size=testset_size)

Applying HeadlineSplitter:   0%|          | 0/93 [00:00<?, ?it/s]           unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to ap

## Show the generated dataset
- We can see the `user_input` and the `reference` = answer.
- It also indicates the `reference_contexts` that was used to generate the answer.

In [11]:
# Extract the dataset and wrap the text
# This is to make the text more readable
# Extract user_input and reference columns and wrap text for readability
import pandas as pd
pd.set_option('display.expand_frame_repr', False)  # Prevents line breaks in DataFrame display
pd.set_option('display.max_colwidth', 300)

df = dataset.to_pandas()[["user_input", "reference", "reference_contexts"]]
df = df.style.set_properties(**{'text-align': 'left'})  # Align text to the left
df

- You can see the `reference_contexts` that was used to generate the answer.
- They can really benefit from better cleaning.